In [ ]:
"""3주차: 선형 회귀 전체 파이프라인 (학생 학습 데이터 버전).

3주차 차시 1~3에서 다루는 선형 회귀(Linear Regression) 실습 코드입니다.

데이터: student_learning_data_kr.csv (학생 학습 활동 → 시험점수 예측)

Contents:
    1. 데이터 로드 및 탐색
    2. 전처리 — 변수 유형 정의 및 표준화
    3. 피처(X)와 타겟(y) 분리
    4. 훈련/테스트 분리
    5. 선형 회귀 모델 생성, 학습, 예측
    6. 회귀 평가지표 계산 (MSE, RMSE, R²)
    7. 학습된 계수 및 절편 해석
    8. 다중공선성 점검
    9. 훈련/테스트 성능 비교
"""

# 교육문제해결형 머신러닝 — 3주차 실습

## 학생 학습 데이터를 활용한 시험점수 예측: 선형 회귀

| 항목 | 내용 |
|------|------|
| **데이터** | `student_learning_data_kr.csv` (학생 학습 데이터 14,003건) |
| **피처(X)** | 주당학습시간, 출석률, 나이, 온라인강좌수, 과제완료율 (수치형 5개) |
| **타겟(y)** | 시험점수 (40~100점, 연속형) — **회귀(Regression)** 문제 |
| **모델** | LinearRegression |
| **평가지표** | MSE · RMSE · R² |
| **핵심 패턴** | `model → fit → predict → score` |

---
## 차시 1 핵심 요약 (이론)

- 선형 회귀 목표: **연속형 숫자(시험점수)** 를 예측하는 것
- 모델 수식: `ŷ = w₁x₁ + w₂x₂ + … + wₙxₙ + b`
- 계수(w): 각 피처가 타겟에 미치는 **영향력**
- 절편(b): 모든 피처가 0일 때의 **기준값**
- 학습 원리: **오차(잔차)를 최소화**하도록 w, b를 조정

| 분류 vs 회귀 | 분류 (1~2주차) | 회귀 (3주차) |
|:---:|:---:|:---:|
| **타겟** | 범주 (A/B/C/D) | 연속 숫자 (점수) |
| **평가** | 정확도 (Accuracy) | MSE · RMSE · R² |
| **예시 모델** | k-NN, DecisionTree | LinearRegression |

---
## Step 1. 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("numpy:", np.__version__)
print("pandas:", pd.__version__)

---
## Step 2. 데이터 로드 및 탐색

- **데이터**: `student_learning_data_kr.csv`
- **피처 (14개 컬럼 중 5개 수치형 사용)**:
  - 주당학습시간, 출석률, 나이, 온라인강좌수, 과제완료율
- **타겟**: `시험점수` (40~100점 범위의 연속형 숫자)

In [ ]:
# CSV 파일 로드
url = "https://raw.githubusercontent.com/SJ-EduLab/ML-for-Education/main/week01/student_learning_data_kr.csv"
df = pd.read_csv(url)

print(f"데이터 shape: {df.shape[0]}행 × {df.shape[1]}열")
print(f"컬럼 목록: {df.columns.tolist()}")
display(df.head())

In [ ]:
# 수치형 변수 기술 통계
print("=== 수치형 변수 기술 통계 ===")
display(df.describe())

---
## Step 3. 데이터 전처리

### 3.1 변수 유형 정의

| 유형 | 변수 |
|------|------|
| 연속형 (수치) | 주당학습시간, 출석률, 나이, 온라인강좌수, 과제완료율, 시험점수 |
| 순서형 | 학습동기, 스트레스수준 |
| 이진형 | 비교과활동, 인터넷접근, 성별, 토론참여 |
| 명목형 | 선호학습유형 |

이번 실습에서는 **수치형 피처 5개**(시험점수 제외)만 사용한다.

In [ ]:
# 변수 유형 정의
continuous = [
    "주당학습시간", "출석률", "나이", "온라인강좌수", "과제완료율", "시험점수",
]
target = "시험점수"

# 피처용 연속형 변수 (타겟 제외)
feature_continuous = [col for col in continuous if col != target]

print(f"피처로 사용할 연속형 변수 (타겟 제외): {feature_continuous}")
print(f"타겟 변수: {target}")

### 3.2 피처 표준화 (StandardScaler)

- `StandardScaler`: 평균=0, 표준편차=1로 변환
- 피처 간 스케일 차이를 제거 → 계수 비교 가능성 향상
- **타겟 변수(시험점수)는 스케일링하지 않음** (원래 점수 단위 유지)

In [ ]:
"""연속형 피처를 StandardScaler로 표준화한다.

타겟 변수(시험점수)는 원래 단위를 유지하기 위해 스케일링 대상에서
제외한다.
"""

scaler = StandardScaler()

# 타겟 제외한 연속형 피처만 스케일링
df[feature_continuous] = scaler.fit_transform(df[feature_continuous])

print("스케일링 후 기술 통계 (피처):")
display(df[feature_continuous].describe().round(2))

print(f"\n⚠ 타겟 변수(시험점수)는 원래 단위 유지:")
print(f"  범위: {df[target].min()} ~ {df[target].max()}점")
print(f"  평균: {df[target].mean():.2f}점")

---
## Step 4. 피처(X)와 타겟(y) 분리

- **피처(X)**: 모델에게 제공하는 입력 (주당학습시간, 출석률, 나이, 온라인강좌수, 과제완료율)
- **타겟(y)**: 모델이 맞춰야 하는 출력 (시험점수)

이번 실습의 타겟은 **시험점수(연속형 숫자)** — 따라서 **회귀** 문제이다.

In [ ]:
# 피처(X)와 타겟(y) 분리
feature_columns = feature_continuous
target_column = target

X = df[feature_columns]
y = df[target_column]

print(f"선택된 피처: {feature_columns}")
print(f"타겟 변수: {target_column}")
print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")

# 결측치 확인
print(f"\n결측치 확인:")
print(X.isnull().sum())
print(f"타겟 결측치: {y.isnull().sum()}")

---
## Step 5. 훈련/테스트 세트 분리

- `test_size=0.3`: 전체의 30%를 테스트 세트로 분리
- `random_state=42`: 재현성(reproducibility)을 위한 랜덤 시드 고정

**왜 나누는가?** → 모델의 **일반화 성능**(새 데이터에서의 진짜 실력)을 측정하기 위해

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"훈련 세트: {X_train.shape[0]:,}개")
print(f"테스트 세트: {X_test.shape[0]:,}개")
print(f"분리 비율: {X_train.shape[0]/len(X)*100:.1f}% 훈련, {X_test.shape[0]/len(X)*100:.1f}% 테스트")

---
## Step 6. 선형 회귀 모델 생성 · 학습 · 예측

scikit-learn 핵심 3줄 패턴 (1주차 복습):
1. `model = LinearRegression()` — 모델 준비
2. `model.fit(X_train, y_train)` — 훈련 데이터로 패턴 학습
3. `model.predict(X_test)` — 새 데이터에 대해 예측

> 분류(k-NN)와 **코드 구조가 동일**하다. 모델 이름만 바뀌었을 뿐이다.

In [ ]:
# 모델 생성 + 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 예측
predictions = model.predict(X_test)

# 예측값과 실제값 비교 (처음 10개)
comparison_df = pd.DataFrame({
    "실제값": y_test.values[:10],
    "예측값": predictions[:10].round(1),
    "오차": (y_test.values[:10] - predictions[:10]).round(1),
})

print("예측 결과 (처음 10개):")
display(comparison_df)

---
## Step 7. 회귀 평가지표 — MSE · RMSE · R²

분류에서는 **정확도(accuracy)** 를 사용했지만,
회귀에서는 타겟이 연속형 숫자이므로 다른 평가지표가 필요하다.

| 지표 | 수식 | 단위 | 해석 |
|------|------|------|------|
| **MSE** | 오차²의 평균 | 점² | 0에 가까울수록 좋음 |
| **RMSE** | √MSE | 점 | 직관적 해석 가능 |
| **R²** | 1 − (모델오차/평균오차) | 비율 | 1에 가까울수록 좋음 |

In [ ]:
"""MSE(평균 제곱 오차)와 RMSE(평균 제곱근 오차)를 계산한다."""

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"\n해석: 평균적으로 약 {rmse:.1f}점 정도 예측이 빗나갑니다.")

In [ ]:
"""R² (결정계수)를 계산한다.

R² = 1 - (모델 오차 / 평균 오차)
- 1에 가까울수록 좋음
- 0이면 평균과 동일한 수준
- 음수이면 평균보다 못함
"""

r2 = r2_score(y_test, predictions)

print(f"R²: {r2:.4f}")
print(f"\n해석: 모델이 시험점수 변동의 약 {r2 * 100:.1f}%를 설명합니다.")

# R² 값 평가
if r2 >= 0.7:
    print("→ 높은 설명력을 가진 좋은 모델입니다.")
elif r2 >= 0.5:
    print("→ 중간 수준의 설명력을 가진 모델입니다.")
elif r2 >= 0.3:
    print("→ 낮은 설명력이지만 피처가 타겟 예측에 어느 정도 도움이 됩니다.")
else:
    print("→ 매우 낮은 설명력입니다. 피처 선택을 재검토할 필요가 있습니다.")

---
## Step 8. 학습된 계수 및 절편 확인

- **계수(w)**: 각 피처가 타겟에 미치는 영향력
- **절편(b)**: 모든 피처가 0(=평균)일 때의 기준값
- ⚠ **주의**: 계수는 "다른 모든 피처를 고정했을 때"의 영향력이다

In [ ]:
"""학습된 모델의 계수와 절편을 확인한다."""

# 계수 DataFrame 생성
coef_df = pd.DataFrame({
    "피처": feature_columns,
    "계수": model.coef_.round(4),
})

print("계수 (w):")
display(coef_df)
print(f"\n절편 (b): {model.intercept_:.4f}")

### 8.1 계수 해석

In [ ]:
"""계수를 해석한다.

피처가 표준화되어 있으므로, 계수의 절대값이 클수록 해당 피처가
시험점수에 미치는 영향이 크다.
"""

# 가장 큰 양수/음수 계수 찾기
max_pos_idx = np.argmax(model.coef_)
min_neg_idx = np.argmin(model.coef_)

print(f"가장 큰 양수 계수:")
print(f"  피처: {feature_columns[max_pos_idx]}")
print(f"  계수: {model.coef_[max_pos_idx]:.4f}")
print(f"  → 다른 조건이 같을 때, {feature_columns[max_pos_idx]} 1단위 증가 시")
print(f"    시험점수가 약 {model.coef_[max_pos_idx]:.2f}점 증가")

print(f"\n가장 큰 음수 계수:")
print(f"  피처: {feature_columns[min_neg_idx]}")
print(f"  계수: {model.coef_[min_neg_idx]:.4f}")
if model.coef_[min_neg_idx] < 0:
    print(f"  → 다른 조건이 같을 때, {feature_columns[min_neg_idx]} 1단위 증가 시")
    print(f"    시험점수가 약 {abs(model.coef_[min_neg_idx]):.2f}점 감소")
else:
    print(f"  → 모든 계수가 양수입니다.")

# 전체 계수 해석
print(f"\n각 피처별 해석:")
print("-" * 60)
for feature, coef in zip(feature_columns, model.coef_):
    direction = "증가" if coef > 0 else "감소"
    print(f"{feature:12s}: {coef:8.4f}  → {feature} ↑ = 시험점수 {direction}")

---
## Step 9. 다중공선성 점검

- **다중공선성**: 피처 간 높은 상관 → 계수 해석이 불안정해짐
- 상관계수 |r| > 0.5인 피처 쌍을 확인
- 예측 성능은 괜찮을 수 있지만, 계수 해석은 위험

In [ ]:
"""피처 간 상관관계를 확인하여 다중공선성을 점검한다."""

# 상관계수 행렬 계산
correlation_matrix = X.corr()

print("상관계수 행렬:")
display(correlation_matrix.round(3))

# 높은 상관관계 찾기 (|r| > 0.5)
threshold = 0.5
print(f"\n상관이 높은 피처 쌍 (|r| > {threshold}):")
print("-" * 50)

high_corr_found = False
for i in range(len(feature_columns)):
    for j in range(i + 1, len(feature_columns)):
        r = correlation_matrix.iloc[i, j]
        if abs(r) > threshold:
            print(f"{feature_columns[i]:12s} ↔ {feature_columns[j]:12s}  r = {r:6.3f}")
            high_corr_found = True

if not high_corr_found:
    print("(없음)")
    print("→ 다중공선성 문제가 심각하지 않습니다.")
else:
    print("\n⚠ 주의: 상관이 높은 피처 쌍의 계수는 개별 해석 시 주의가 필요합니다.")
    print("  데이터가 조금만 바뀌어도 계수 값이 크게 변할 수 있습니다.")

---
## Step 10. 훈련/테스트 성능 비교

- 훈련 성능: 이미 본 데이터에 대한 성능 (가짜 실력일 수 있음)
- 테스트 성능: 본 적 없는 데이터에 대한 성능 (진짜 실력)

| 패턴 | 훈련 | 테스트 | 진단 |
|------|------|--------|------|
| 양호 | 좋음 | 좋음 (비슷) | ✓ |
| 과적합 | 좋음 | 나쁨 (gap 큼) | ⚠ |
| 과소적합 | 나쁨 | 나쁨 | ✗ |

In [ ]:
"""훈련 세트와 테스트 세트의 성능을 비교한다."""

# 훈련 세트 성능
train_predictions = model.predict(X_train)
train_mse = mean_squared_error(y_train, train_predictions)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, train_predictions)

# 테스트 세트 성능
test_mse = mean_squared_error(y_test, predictions)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, predictions)

# 결과 출력
performance_df = pd.DataFrame({
    "MSE": [train_mse, test_mse],
    "RMSE": [train_rmse, test_rmse],
    "R²": [train_r2, test_r2],
}, index=["훈련 세트", "테스트 세트"])

print("성능 비교:")
display(performance_df.round(4))

# 일반화 성능 분석
print(f"\n분석:")
if train_mse < test_mse:
    diff_percent = (test_mse - train_mse) / train_mse * 100
    print(f"훈련 MSE < 테스트 MSE (차이: {diff_percent:.1f}%)")
    if diff_percent < 10:
        print("→ 정상적인 패턴. 일반화 성능이 양호합니다.")
    elif diff_percent < 30:
        print("→ 약간의 과적합 경향이 있을 수 있습니다.")
    else:
        print("→ 과적합 가능성이 있습니다. 모델 복잡도 검토가 필요합니다.")
else:
    print("훈련 MSE ≥ 테스트 MSE")
    print("→ 데이터 분할에 따라 발생 가능한 상황입니다.")

---
## Step 11. 최종 요약

In [ ]:
"""실습 결과를 요약한다."""

print("=" * 50)
print("  3주차 선형 회귀 실습 — 최종 요약")
print("=" * 50)

print(f"\n데이터셋 정보")
print(f"  전체 샘플: {len(df):,}개")
print(f"  훈련 세트: {len(X_train):,}개")
print(f"  테스트 세트: {len(X_test):,}개")
print(f"  피처 개수: {len(feature_columns)}개")

print(f"\n타겟 변수")
print(f"  시험점수 ({df[target].min()}~{df[target].max()}점)")
print(f"  평균: {df[target].mean():.2f}점")

print(f"\n모델 성능 (테스트 세트)")
print(f"  RMSE: {test_rmse:.2f}점")
print(f"  R²:   {test_r2:.4f} ({test_r2 * 100:.1f}% 설명력)")

max_pos_idx = np.argmax(model.coef_)
print(f"\n주요 발견")
print(f"  가장 큰 영향: {feature_columns[max_pos_idx]} (계수: {model.coef_[max_pos_idx]:.4f})")

---
## 학습 정리

| # | 핵심 | 설명 |
|---|------|------|
| 1 | **회귀 평가지표** | MSE (오차²) · RMSE (√MSE, 직관적) · R² (설명력) |
| 2 | **계수 해석** | 부호 = 방향, 크기 = "다른 피처 고정 시" 영향력 |
| 3 | **다중공선성** | 피처 간 높은 상관 → 계수 불안정 (예측은 괜찮을 수 있음) |
| 4 | **일반화 성능** | 항상 테스트 세트로 최종 평가! |
| 5 | **scikit-learn 패턴** | 분류(k-NN)와 동일한 `fit → predict → score` 구조 |

### 주의사항
1. 스케일이 다른 피처의 계수는 직접 비교 불가 → **표준화 후 비교**
2. 상관이 높은 피처 쌍의 계수 해석 주의 → **다중공선성 점검 필수**
3. 이 결과는 `random_state=42`에서의 단일 분할 결과이다
   → 더 안정적인 평가: **4주차 교차검증(Cross-Validation)**